In [96]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import random

In [97]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import random

#description d'un agent : nom:(opinion, deja_vu , force_de_persuasion , conviction)
#                         nom:(  op   , deja_vu ,       fp            ,     p     )

#VARIABLES GLOBALES / PARAMETRES DU MODELE :
N_1dim = 5                #nb d'agents sur une dimension
N = N_1dim**2             #nb d'agents total
nb_om = 10                 #nb d'organismes médiatiques (malveillants ou non)
nb_xom = 4                #nb d'organismes médiatiques malveillants
mal = nb_xom/nb_om        #degré de malveillance (= proportion de médias malveillants)
co = 7                    #degré de connectivité de l'environnement (= nb de réseaux moyen par agent = somme(nb_res) / nb_agents)
steps = 100               #nb de pas de temps
news = 0.5 #crédibilité de la news
seuil = 0.7               #seuil

print('Crédibilité de la news :',news,'\n')

#Initialisation d'un réseau d'humains :
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Noms_reseau_discu=[str(k) for k in range(1,N+1)]

#Initialisation d'un réseau avec des agents malveillants (temporaire)
nb_malv=6
def init_sys_avec_malv(n_1dim,nb_malv):
    
    Pop={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
    L=np.random.choice(range(1,(n_1dim**2)+1), size=nb_malv, replace=False)
    for e in L:
        Pop[str(e)]=(1,0,0.9,0.8)
    
    return Pop

Pop_malv=init_sys_avec_malv(N_1dim,nb_malv)
print('Réseau avec malveillants : \n',Pop_malv,'\n')

#Initialisation d'un système avec organismes médiatiques (malveillants et neutres):
List_om=np.random.choice(range(1,N+1), size=nb_om, replace=False)
print('Liste des médias :',List_om,'\n')

List_xom=np.random.choice(List_om, size=nb_xom, replace=False)
print('Liste des médias malveillants :',List_xom,'\n')

def init_ac_om_et_xom(reseau,list_om,list_xom):
    for name in reseau:
        if 'O' not in name and 'X' not in name:
            if int(name) in list_om and int(name) not in list_xom:
                reseau['OM'+str(name)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.5)
                del reseau[name]
            if int(name) in list_xom:
                reseau['XOM'+str(name)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.75)
                del reseau[name]
                
    return reseau


Crédibilité de la news : 0.5 

Réseau avec malveillants : 
 {'1': (1, 0, 0.9, 0.8), '2': (0, 0, 0.3913324348920182, 0.5), '3': (0, 0, 0.15691336343561701, 0.5), '4': (0, 0, 0.1035682149510917, 0.5), '5': (1, 0, 0.9, 0.8), '6': (0, 0, 0.08940720296023097, 0.5), '7': (0, 0, 0.7920084322475955, 0.5), '8': (0, 0, 0.02383822810288594, 0.5), '9': (0, 0, 0.5635870343657617, 0.5), '10': (0, 0, 0.9085474234444638, 0.5), '11': (0, 0, 0.5704637397954168, 0.5), '12': (0, 0, 0.4530642167616571, 0.5), '13': (0, 0, 0.3894206683578303, 0.5), '14': (0, 0, 0.45416183221664974, 0.5), '15': (0, 0, 0.9241435427720658, 0.5), '16': (0, 0, 0.5620067841761547, 0.5), '17': (0, 0, 0.03544295237983153, 0.5), '18': (0, 0, 0.6209343424526058, 0.5), '19': (1, 0, 0.9, 0.8), '20': (1, 0, 0.9, 0.8), '21': (0, 0, 0.4651856478046056, 0.5), '22': (1, 0, 0.9, 0.8), '23': (1, 0, 0.9, 0.8), '24': (0, 0, 0.767250532410178, 0.5), '25': (0, 0, 0.5652911005087073, 0.5)} 

Liste des médias : [17  6  3 15 11 21 22 18 14 23] 

List

In [98]:
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Population avec médias : \n',Pop_ac_om,'\n')

#Réseau à valeurs quelconques (au milieu de la simulation par exemple) pour faire des tests :
Test_reseau={str(k):(np.random.choice([0,1]),np.random.choice(range(steps)),np.random.random_sample(),np.random.random_sample()) for k in range(1,N+1)}

#Liste/matrice des noms des agents :
    #Matrice sans médias
Noms_agents_sans=[name for name in Pop_malv]
np.random.shuffle(Noms_agents_sans)
Mat_pop_sans_om=np.reshape(Noms_agents_sans,(N_1dim,N_1dim)).copy()
print('Matrice agents (sans médias) :\n',Mat_pop_sans_om,'\n')
    #Matrice avec médias
Noms_agents_avec=[name for name in Pop_ac_om]
np.random.shuffle(Noms_agents_avec)
Mat_pop_ac_om=np.reshape(Noms_agents_avec,(N_1dim,N_1dim)).copy()
print('Matrice agents (avec médias) :\n',Mat_pop_ac_om,'\n')

#Matrice des opinions de départ :
def matrice_opinions(reseau,names,n_1dim):
    Op=np.zeros((n_1dim,n_1dim))
    i=0
    j=0
    for nom in names:
        op,deja,fp,p=reseau[nom]
        Op[i,j]=op
        if j<(n_1dim-1):
            j=j+1
        elif j==(n_1dim-1):
            j=0
            i=i+1
    return Op

print('Matrice op (avec médias) :\n',matrice_opinions(Pop_ac_om,Noms_agents_avec,N_1dim),'\n')

#Matrice des convictions de départ
def matrice_convic(reseau,names,n_1dim):
    P=np.zeros((n_1dim,n_1dim))
    i=0
    j=0
    for nom in names:
        op,deja,fp,p=reseau[nom]
        P[i,j]=p
        if j<(n_1dim-1):
            j=j+1
        elif j==(n_1dim-1):
            j=0
            i=i+1
    return P

print('Matrice p (avec médias) :\n',matrice_convic(Pop_ac_om,Noms_agents_avec,N_1dim),'\n')

Population avec médias : 
 {'1': (0, 0, 0.6015633948474909, 0.5), '2': (0, 0, 0.8094301092061899, 0.5), '4': (0, 0, 0.8584371515788264, 0.5), '5': (0, 0, 0.6434867738267376, 0.5), '7': (0, 0, 0.33564643579856046, 0.5), '8': (0, 0, 0.2528142404971352, 0.5), '9': (0, 0, 0.35452126159338004, 0.5), '10': (0, 0, 0.18560266578690388, 0.5), '12': (0, 0, 0.8070450615430039, 0.5), '13': (0, 0, 0.4869451929300631, 0.5), '16': (0, 0, 0.4148741532582215, 0.5), '19': (0, 0, 0.03873923136249824, 0.5), '20': (0, 0, 0.6549303411257049, 0.5), '24': (0, 0, 0.9196869186665231, 0.5), '25': (0, 0, 0.35109335323509505, 0.5), 'OM3': (0, 0, 0.7038881495729553, 0.5), 'OM6': (0, 0, 0.9657761879948759, 0.5), 'XOM11': (0, 0, 0.8141528295921805, 0.75), 'XOM14': (0, 0, 0.9864308435232472, 0.75), 'OM15': (0, 0, 0.7030234830178687, 0.5), 'OM17': (0, 0, 0.9589214624526425, 0.5), 'OM18': (0, 0, 0.9268175408604137, 0.5), 'OM21': (0, 0, 0.9840617617599712, 0.5), 'XOM22': (0, 0, 0.8503760716562734, 0.75), 'XOM23': (0, 0, 

In [99]:
#RESEAUX SOCIAUX
L_reseaux=['facebook','qzone','instagram','tumblr','twitter','baidu','weibo','snapchat','pinterest','linkedin']

#Nb de réseaux par agent:
def nb_res(co,n):
    k=0
    L=[]

    while k<n-1:
        if k%2==0:
            if co<5:
                nb=np.random.choice(range(2*co+1))
            else:
                nb=np.random.choice(range(11))
            L.append(nb)
        else:
            if 2*co-nb<=10:
                L.append(2*co-nb)
            else:
                L[k-1]=L[k-1]+(2*co-nb-10)
                L.append(10)
        k=k+1
    if n%2==1:
        L.append(co)
    else:
        if 2*co-nb<=10:
            L.append(2*co-nb)
        else:
            L[k-1]=L[k-1]+(2*co-nb-10)
            L.append(10)
        
    return L

Nb_res={name : nb for (name,nb) in zip(Noms_agents_avec,nb_res(co,N))}
print('Nb de réseaux par agent :\n',Nb_res,'\n')

#Agents et les réseaux auxuels ils appartiennent:
Agents_ac_reseaux={name : list(np.random.choice(L_reseaux, size=Nb_res[name], replace=False)) for name in Noms_agents_avec}
print('Agents et les réseaux auxquels ils appartiennent :\n',Agents_ac_reseaux,'\n')

#Réseaux et leurs utilisateurs:
def res_et_ag(ag_res,reseaux):
    D=dict()
    for res in reseaux:
        D[res]=[]
        for ag in ag_res:
            if res in ag_res[ag]:
                D[res].append(ag)
    return D
    
Reseaux_ac_agents=res_et_ag(Agents_ac_reseaux,L_reseaux)
print('Reseaux et leurs utilisateurs :\n',Reseaux_ac_agents,'\n')

#Réseau virtuel (dico liste amis)
def reseau_virtu(agents_et_res,res_et_agents,nb_res):
    D=dict()
    for a in agents_et_res:
        if nb_res[a]==0:
            D[a]=[]
        else:
            for r in agents_et_res[a]:
                Amis=set(np.random.choice(res_et_agents[r],size=np.random.choice(range(len(res_et_agents[r])))))
                if a in Amis:
                    Amis.remove(a)
                D[a]=list(Amis)    
    for a in D:
        for ami in D[a]:
            if a not in D[ami] and nb_res[ami]!=0:
                D[ami].append(a)
    return D

Reseau_virtuel=reseau_virtu(Agents_ac_reseaux,Reseaux_ac_agents,Nb_res)
print('Réseau virtuel :\n',Reseau_virtuel)

Nb de réseaux par agent :
 {'8': 5, 'OM21': 9, 'OM3': 4, 'XOM11': 10, 'OM18': 7, '19': 7, '20': 4, 'XOM22': 10, 'OM15': 4, '24': 10, '13': 7, '12': 7, '1': 7, '10': 7, '7': 8, '25': 6, 'XOM14': 5, '9': 9, '4': 8, '16': 6, 'OM17': 4, 'OM6': 10, 'XOM23': 4, '5': 10, '2': 7} 

Agents et les réseaux auxquels ils appartiennent :
 {'8': ['facebook', 'qzone', 'tumblr', 'instagram', 'pinterest'], 'OM21': ['baidu', 'linkedin', 'pinterest', 'facebook', 'qzone', 'tumblr', 'snapchat', 'twitter', 'instagram'], 'OM3': ['facebook', 'tumblr', 'linkedin', 'qzone'], 'XOM11': ['baidu', 'qzone', 'twitter', 'linkedin', 'snapchat', 'instagram', 'tumblr', 'pinterest', 'facebook', 'weibo'], 'OM18': ['linkedin', 'instagram', 'tumblr', 'weibo', 'snapchat', 'qzone', 'twitter'], '19': ['tumblr', 'baidu', 'pinterest', 'snapchat', 'facebook', 'instagram', 'weibo'], '20': ['linkedin', 'twitter', 'facebook', 'tumblr'], 'XOM22': ['snapchat', 'linkedin', 'tumblr', 'qzone', 'baidu', 'twitter', 'facebook', 'pinterest', '

In [100]:
#Lancement fake news
#choix aléatoire d'une média malveillant de la liste_xom comme le départ de la propagation du fake news

def lancement_fake_news(pop,list_xom):
    lanceur='XOM'+str(np.random.choice(list_xom))
    pop[lanceur]=(1,1,np.random.uniform(low=0.7,high=1.0),0.75)
    for xom in list_xom:
        if 'XOM'+str(xom)!=lanceur:
            pop['XOM'+str(xom)]=(0,0,np.random.uniform(low=0.7,high=1.0),0.75)
    return pop,lanceur

Pop_ap_lan,Lanceur=lancement_fake_news(Pop_ac_om, List_xom)
print('Lanceur :',Lanceur)
print('Population après le lancement du fake news : \n',Pop_ap_lan)

Lanceur : XOM22
Population après le lancement du fake news : 
 {'1': (0, 0, 0.6015633948474909, 0.5), '2': (0, 0, 0.8094301092061899, 0.5), '4': (0, 0, 0.8584371515788264, 0.5), '5': (0, 0, 0.6434867738267376, 0.5), '7': (0, 0, 0.33564643579856046, 0.5), '8': (0, 0, 0.2528142404971352, 0.5), '9': (0, 0, 0.35452126159338004, 0.5), '10': (0, 0, 0.18560266578690388, 0.5), '12': (0, 0, 0.8070450615430039, 0.5), '13': (0, 0, 0.4869451929300631, 0.5), '16': (0, 0, 0.4148741532582215, 0.5), '19': (0, 0, 0.03873923136249824, 0.5), '20': (0, 0, 0.6549303411257049, 0.5), '24': (0, 0, 0.9196869186665231, 0.5), '25': (0, 0, 0.35109335323509505, 0.5), 'OM3': (0, 0, 0.7038881495729553, 0.5), 'OM6': (0, 0, 0.9657761879948759, 0.5), 'XOM11': (0, 0, 0.9719575044708328, 0.75), 'XOM14': (0, 0, 0.8275180520393992, 0.75), 'OM15': (0, 0, 0.7030234830178687, 0.5), 'OM17': (0, 0, 0.9589214624526425, 0.5), 'OM18': (0, 0, 0.9268175408604137, 0.5), 'OM21': (0, 0, 0.9840617617599712, 0.5), 'XOM22': (1, 1, 0.89562

In [101]:
#Influence crébibilité news sur le p d'une personne

def influ_credib_news(news,p,fp):
    #news: crédibilité de la news
    i = p+(1-fp)*(news-p)
    return i

print(influ_credib_news(news,0.5,0.1),'\n')

def influ_partage(pop,res_virtu,partageur): #pop to get op,deja,fp,p ; res_virtu to get liste partage

    Liste_partage=res_virtu[partageur]
    op_p,deja_p,fp_p,p_p=pop[partageur]
    
    if deja_p==0:
        return pop
    
    P=[]   #liste des convictions des membres de la liste partage
    Fp=[]  #liste des forces de persuasion des membres de la liste partage
    i=0
    
    for k in Liste_partage:
        op_k,deja_k,fp_k,p_k=pop[k]
        P.append(p_k)
        Fp.append(fp_k)
        
    for a in Liste_partage:
        op_a,deja_a,fp_a,p_a=pop[a]
        if 'X' not in a:
            if fp_p>fp_a:
                p_a=p_a+fp_p*(p_p-p_a)
            p_a=influ_credib_news(news,p_a,fp_a)
        if op_p==1:
            deja_a=deja_a+1
        pop[a]=op_a,deja_a,fp_a,p_a
    
    return pop

Partageur=np.random.choice(Noms_agents_avec)
print('Pop avant partage \n',Pop_ac_om,'\n')
print('Celui qui partage l\'info :', Partageur,'\n')
print('Amis de celui qui partage :', Reseau_virtuel[Partageur],'\n')
print('Pop après partage \n', influ_partage(Pop_ac_om, Reseau_virtuel, Partageur))

0.5 

Pop avant partage 
 {'1': (0, 0, 0.6015633948474909, 0.5), '2': (0, 0, 0.8094301092061899, 0.5), '4': (0, 0, 0.8584371515788264, 0.5), '5': (0, 0, 0.6434867738267376, 0.5), '7': (0, 0, 0.33564643579856046, 0.5), '8': (0, 0, 0.2528142404971352, 0.5), '9': (0, 0, 0.35452126159338004, 0.5), '10': (0, 0, 0.18560266578690388, 0.5), '12': (0, 0, 0.8070450615430039, 0.5), '13': (0, 0, 0.4869451929300631, 0.5), '16': (0, 0, 0.4148741532582215, 0.5), '19': (0, 0, 0.03873923136249824, 0.5), '20': (0, 0, 0.6549303411257049, 0.5), '24': (0, 0, 0.9196869186665231, 0.5), '25': (0, 0, 0.35109335323509505, 0.5), 'OM3': (0, 0, 0.7038881495729553, 0.5), 'OM6': (0, 0, 0.9657761879948759, 0.5), 'XOM11': (0, 0, 0.9719575044708328, 0.75), 'XOM14': (0, 0, 0.8275180520393992, 0.75), 'OM15': (0, 0, 0.7030234830178687, 0.5), 'OM17': (0, 0, 0.9589214624526425, 0.5), 'OM18': (0, 0, 0.9268175408604137, 0.5), 'OM21': (0, 0, 0.9840617617599712, 0.5), 'XOM22': (1, 1, 0.8956266353345248, 0.75), 'XOM23': (0, 0, 0

In [102]:
#Modification du paramètre p de x en fonction de la force de persuasion de y et vice versa :
def force_persu(x, y, reseau):
        
    opx,deja_vux,fpx,px=reseau[str(x)]
    opy,deja_vuy,fpy,py=reseau[str(y)]
    if fpx>fpy:
        py=py+fpx*(px-py)
    elif fpy>fpx:
        px=px+fpy*(py-px)
    
    reseau[str(x)]=(opx,deja_vux+1,fpx,px)
    reseau[str(y)]=(opy,deja_vuy+1,fpy,py)
    
    return reseau


#Influence du voisinage :
    # si 2 personnes, force persu. Sinon, moyenne des p pondérée des fp; si abs(pi-moy)>seuil, pi=pi, sinon pi=moy
def influ_voisinage(list_discu,reseau,seuil):
    
    P=[]   #liste des convictions des membres de la discussion
    F=[]   #liste des forces de persuasion des membres de la discussion
    N=[]   #liste des noms des membres de la discussion
    i=0
    
    for k in list_discu:
        op_k,deja_k,fp_k,p_k=reseau[k]
        N.append(k)
        P.append(p_k)
        F.append(fp_k)
            
    if len(N)==2:
        reseau=force_persu(N[0],N[1],reseau)
    else:
        for a in N:
            op_a,deja_a,fp_a,p_a=reseau[a]
            other_p=P[0:i]+P[i+1:len(P)]
            other_fp=F[0:i]+F[i+1:len(F)]            
            moy=np.average(other_p,weights=other_fp)
            if abs(p_a-moy)<seuil:
                p_a=moy
            if moy!=0.5:
                deja_a=deja_a+1
            reseau[a]=(op_a,deja_a,fp_a,p_a)
            i=i+1
            
    return reseau

print('Réseau avant :',Test_reseau,'\n Discussion :',List_discu)
print('Réseau après :',influ_voisinage(List_discu,Test_reseau,seuil))

Réseau avant : {'1': (0, 19, 0.5133778007284251, 0.3257611263049601), '2': (1, 82, 0.39319365394230055, 0.8009570128939861), '3': (1, 26, 0.6155316381615988, 0.5444440677205051), '4': (0, 45, 0.803501184435564, 0.22638572156152204), '5': (1, 78, 0.10446951593843357, 0.5929413757068572), '6': (0, 92, 0.13851650089739453, 0.2442415006858467), '7': (1, 16, 0.9856442270197802, 0.373904917911668), '8': (0, 79, 0.776449694022063, 0.948190583758621), '9': (0, 11, 0.37563034491264036, 0.049383143549405206), '10': (0, 7, 0.9121063328372976, 0.09361162564366843), '11': (1, 56, 0.6561982243954856, 0.5415132795187063), '12': (1, 69, 0.26290289652897736, 0.5239899202745524), '13': (0, 84, 0.38332790248105597, 0.012203220418714578), '14': (1, 46, 0.272275783406169, 0.48762111202269), '15': (1, 27, 0.029875048385979097, 0.9214656374420134), '16': (0, 8, 0.4917356977277788, 0.9156778315933204), '17': (1, 16, 0.9283372319395374, 0.6128109508689074), '18': (0, 52, 0.014469143331907408, 0.320779198422866

In [103]:
# Sélection d'un agent et de son voisinage pour établir un discussion
def selection_voisinage(Matrice,n_1dim,a_central):
    '''Sélection aléatoire des voisinages d'un individu
    '''
    L=[]                               #liste voisinage (pers susceptibles de discuter)
    Ldiscu=[]                          #liste discussion (pers qui vont discuter)
    size=np.random.choice(range(2,10)) #nb pers discussion
    
    ic=np.argwhere(Matrice==a_central)[0][0] #ic,jc : indices de l'agent central
    jc=np.argwhere(Matrice==a_central)[0][1]
    
    if ic>0 and ic<(n_1dim-1) and jc>0 and jc<(n_1dim-1):
        L=np.reshape(Matrice[ic-1 : ic+2 , jc-1 : jc+2] , 9)        
    elif ic==0 and jc>0 and jc<(n_1dim-1):          #agent central sur arête sup
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic+1,jc-1 : jc+2])
           ,(Matrice[n_1dim-1, jc-1 : jc+2])]
        L=np.reshape(L,9)
    elif ic==(n_1dim-1) and jc>0 and jc<(n_1dim-1): #agent central sur arête inf
        L=[(Matrice[ic,jc-1 : jc+2])
           ,(Matrice[ic-1,jc-1 : jc+2])
           ,(Matrice[0, jc-1 : jc+2])]
        L=np.reshape(L,9)        
    elif jc==0 and ic>0 and ic<(n_1dim-1):          #agent central sur arête gauche
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc+1])
           ,(Matrice[ic-1 : ic+2,n_1dim-1])]
        L=np.reshape(L,9)        
    elif jc==(n_1dim-1) and ic>0 and ic<(n_1dim-1): #agent central sur arête droite
        L=[(Matrice[ic-1 : ic+2,jc])
           ,(Matrice[ic-1 : ic+2,jc-1])
           ,(Matrice[ic-1 : ic+2,0])]
        L=np.reshape(L,9)        
    elif ic==0 and jc==0:                           #agent central sur coin sup/gauche
        L=[Matrice[0,n_1dim-1],Matrice[0,0],Matrice[0,1]
           ,Matrice[1,n_1dim-1],Matrice[1,0],Matrice[1,1]
           ,Matrice[n_1dim-1,0],Matrice[n_1dim-1,1],Matrice[n_1dim-1,n_1dim-1]]
    elif ic==0 and jc==n_1dim-1:                    #agent central sur coin sup/droit
        L=[Matrice[ic,jc-1],Matrice[ic,jc],Matrice[ic,0]
          ,Matrice[ic+1,jc-1],Matrice[ic+1,jc],Matrice[ic+1,0]
          ,Matrice[n_1dim-1,jc-1],Matrice[n_1dim-1,jc],Matrice[n_1dim-1,0]]
    elif ic==n_1dim-1 and jc==0:                    #agent central sur coin inf/gauche
        L=[Matrice[ic-1,n_1dim-1],Matrice[ic-1,jc],Matrice[ic-1,jc+1]
          ,Matrice[ic,n_1dim-1],Matrice[ic,jc],Matrice[ic,jc+1]
          ,Matrice[0,n_1dim-1],Matrice[0,0],Matrice[0,1]]
    elif ic==n_1dim-1 and jc==n_1dim-1:             #agent central sur coin inf/droit
        L=[Matrice[ic-1,jc-1],Matrice[ic-1,jc],Matrice[ic-1,0]
          ,Matrice[ic,jc-1],Matrice[ic,jc],Matrice[ic,0]
          ,Matrice[0,jc-1],Matrice[0,jc],Matrice[0,0]]
    
    Ldiscu=np.random.choice(L,size,replace=False)
             
    return Ldiscu

Population=Mat_pop_sans_om
List_discu=selection_voisinage(Population,N_1dim,np.random.choice(Noms_reseau_discu))
print('Population :\n',Population)
print('Liste des personnes qui vont discuter :',List_discu)

Population :
 [['24' '11' '9' '15' '10']
 ['3' '19' '2' '5' '8']
 ['18' '23' '12' '4' '16']
 ['1' '13' '7' '17' '20']
 ['14' '21' '22' '25' '6']]
Liste des personnes qui vont discuter : ['12' '2']


In [104]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simu_p_finale(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):

    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_convic(pop,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
            Res_p_changé=influ_partage(pop,res_virtu,i)          
        else:
            i=np.random.choice(names)
            k=np.random.choice([0,1])
            if k==0:
                list_discu=selection_voisinage(mat_pop,n_1dim,i)
                for a in list_discu:
                    if 'O' in a:
                        list_discu=list_discu[list_discu!=a]
                if len(list_discu)>=2:
                    Res_p_changé=influ_voisinage(list_discu,pop,seuil)
            else:
                Res_p_changé=influ_partage(pop,res_virtu,i)          

        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        P=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            P.append(p)
        P_final=np.reshape(P, (n_1dim,n_1dim))
        results.append(P_final.copy())
    return results

Simu_p_finale=simu_p_finale(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simu_p_finale

Matrice population :
 [['8' 'OM21' 'OM3' 'XOM11' 'OM18']
 ['19' '20' 'XOM22' 'OM15' '24']
 ['13' '12' '1' '10' '7']
 ['25' 'XOM14' '9' '4' '16']
 ['OM17' 'OM6' 'XOM23' '5' '2']]
Lanceur : XOM11
Liste d'abonnés du lanceur : ['12', '25', 'OM18', '10', '16', 'OM6', 'XOM23', '2']


[array([[0.5 , 0.5 , 0.5 , 0.75, 0.5 ],
        [0.5 , 0.5 , 0.75, 0.5 , 0.5 ],
        [0.5 , 0.5 , 0.5 , 0.5 , 0.5 ],
        [0.5 , 0.75, 0.5 , 0.5 , 0.5 ],
        [0.5 , 0.5 , 0.75, 0.5 , 0.5 ]]),
 array([[0.5       , 0.5       , 0.5       , 0.75      , 0.67275585],
        [0.5       , 0.5       , 0.75      , 0.5       , 0.5       ],
        [0.5       , 0.51489671, 0.5       , 0.54444718, 0.5       ],
        [0.56678294, 0.75      , 0.5       , 0.5       , 0.50405154],
        [0.5       , 0.5       , 0.75      , 0.5       , 0.63933445]]),
 array([[0.5       , 0.5       , 0.5       , 0.75      , 0.67275585],
        [0.5       , 0.5       , 0.75      , 0.5       , 0.5       ],
        [0.5       , 0.51489671, 0.5       , 0.54444718, 0.5       ],
        [0.56678294, 0.75      , 0.5       , 0.5       , 0.50405154],
        [0.5       , 0.5       , 0.75      , 0.5       , 0.63933445]]),
 array([[0.5       , 0.5       , 0.5       , 0.75      , 0.67275585],
        [0.5       , 0.5

In [106]:
def moyenne():
    L=[]
    O= Simu_p_finale[-1].ravel()
    print('O:',O)
    for e in O:
        if e!=0.75:
            L.append(e)
            moyenne= np.sum(L)/len(L)
    return moyenne

print('Moyenne des convictions à la fin:',moyenne())

O: [0.56575407 0.5        0.51286839 0.75       0.67503835 0.50020996
 0.50093616 0.75       0.52298598 0.51078713 0.52399199 0.51495501
 0.50678298 0.54742274 0.63677573 0.52141984 0.75       0.50423646
 0.50784359 0.50421975 0.5        0.5        0.75       0.50262163
 0.59378303]
Moyenne des convictions à la fin: 0.5310777519107598


In [107]:
Reseau_discu={str(k):(0,0,fp,0.5) for k in range(1,N+1) for fp in np.random.uniform(low=0.0, high=1.0,size=N)}
Pop_ac_om=init_ac_om_et_xom(Reseau_discu,List_om,List_xom)
print('Matrice population :\n',Mat_pop_ac_om)

def simu_op_finale(pop,res_virtu,names,n_1dim,steps,list_xom,mat_pop):

    pop_ap_lan,lanceur=lancement_fake_news(pop,list_xom)
    results=[matrice_opinions(pop,names,n_1dim)]
    print('Lanceur :',lanceur)
    print('Liste d\'abonnés du lanceur :',res_virtu[lanceur])
    
    for simu in range(steps):
        if simu==0:
            i=lanceur
            Res_p_changé=influ_partage(pop,res_virtu,i)          
        else:
            i=np.random.choice(names)
            if simu%2==1:
                list_discu=selection_voisinage(mat_pop,n_1dim,i)
                for a in list_discu:
                    if 'O' in a:
                        list_discu=list_discu[list_discu!=a]
                if len(list_discu)>=2:
                    Res_p_changé=influ_voisinage(list_discu,pop,seuil)
            else:
                Res_p_changé=influ_partage(pop,res_virtu,i)          

        for c in Res_p_changé:
            op,deja,fp,p=Res_p_changé[c]
            if p>0.5 and op==0:
                op=1
                Res_p_changé[c]=(op,deja,fp,p)
            elif p<0.5 and op==1:
                op=0
                Res_p_changé[c]=(op,deja,fp,p)
        Op=[]
        for e in mat_pop.ravel():
            op,deja,fp,p=Res_p_changé[e]
            Op.append(op)
        Op_final=np.reshape(Op, (n_1dim,n_1dim))
        results.append(Op_final.copy())
    return results

Simu_op_finale=simu_op_finale(Pop_ac_om,Reseau_virtuel,Noms_agents_avec,N_1dim,steps,List_xom,Mat_pop_ac_om)
Simu_op_finale

Matrice population :
 [['8' 'OM21' 'OM3' 'XOM11' 'OM18']
 ['19' '20' 'XOM22' 'OM15' '24']
 ['13' '12' '1' '10' '7']
 ['25' 'XOM14' '9' '4' '16']
 ['OM17' 'OM6' 'XOM23' '5' '2']]
Lanceur : XOM22
Liste d'abonnés du lanceur : ['16', '8', '13', '10', '7', 'XOM23']


[array([[0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0]]), array([[1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1],
        [0, 1, 1, 

In [108]:
def taux_final():
    compte = 0
    Y=Simu_op_finale[-1].ravel()
    print('Y:',Y)
    for x in Y:
        if x==1:
            compte = compte + 1
            taux_f=compte/len(Y)
    return taux_f*100

print('Taux d\'opinion à 1 à la fin (%):', taux_final())

Y: [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Taux d'opinion à 1 à la fin (%): 96.0


In [109]:
def tps_adoption():
    for simu in range(steps):
        taux=taux_final()
        if taux>=75.0:
            return simu

print(tps_adoption())

Y: [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0
